<a href="https://colab.research.google.com/github/cagBRT/Intro-to-Pandas/blob/master/Pandas_Performance_Enhancement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook covers some techniques for speeding up performance for large datasets (over 1 million rows)

In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/cagBRT/Intro-to-Pandas.git cloned-repo
%cd cloned-repo
!ls

1. Download the dataset here:

In [ ]:
#https://www.kaggle.com/competitions/tabular-playground-series-sep-2021/rules

2. It will be be in zip format. <br>
Unzip the folder

3. Upload using file upload:<br>
>tabular-playground-series-sep-2021/train.csv<br>
>tabular-playground-series-sep-2021/test.csv


4. Wait for the upload to finish. <br>
This can take about 25 minutes

In [ ]:
import pandas as pd
import numpy as np

## Use replace to replace specific values

While speed is the first benefit of replace, the second is its flexibility.

We can replace all question marks with NaN - an operation that would take multiple calls with index-based replacement.

Nested replacement helps when you only want to affect the values of specific columns. Here, we are replacing values only in education and income columns.



In [ ]:
s = pd.Series([1, 2, 3, 4, 5])
s.replace(1, 5)

In [ ]:
s.replace([1, 2], method='bfill')

In [ ]:
df = pd.DataFrame({'A': [0, 1, 2, 3, 4],
                   'B': [5, 6, 7, 8, 9],
                   'C': ['a', 'b', 'c', 'd', 'e']})
df.replace(0, 5)

In [ ]:
df.replace([0, 1, 2, 3], 4)

In [ ]:
df.replace([0, 1, 2, 3], [4, 3, 2, 1])

In [ ]:
adult_income = pd.read_csv("adult.csv")

In [ ]:
adult_income.shape

In [ ]:
adult_income.columns

In [ ]:
adult_income.isna().sum()

In [ ]:
adult_income.replace(to_replace="?", value=np.nan, inplace=True)

In [ ]:
adult_income.isna().sum()

replace allows using lists or dictionaries to change multiple values simultaneously:



In [ ]:
adult_income.gender.value_counts()

In [ ]:
adult_income.replace(["Male", "Female"], ["M", "F"], inplace=True)

In [ ]:
adult_income.gender.value_counts()

When replacing a list of values with another, they will have a one-to-one, index-to-index mapping.

In [ ]:
adult_income["native-country"].value_counts()

In [ ]:
adult_income.replace({"United States": "USA", "US": "USA"}, inplace=True)

In [ ]:
adult_income["native-country"].value_counts()

In [ ]:
adult_income.replace({"United States": "USA", "US": "USA"}, inplace=True)

In [ ]:
adult_income["native-country"].value_counts()

In [ ]:
adult_income.education.value_counts()

In [ ]:
adult_income.income.value_counts()

In [ ]:
adult_income.replace(
    {
        "education": {"HS-grad": "High school", "Some-college": "College", "12th":"High school"},
        "income": {"<=50K": 0, ">50K": 1},
    },
    inplace=True,
)

In [ ]:
adult_income.education.value_counts()

In [ ]:
adult_income.income.value_counts()

# Iterating efficiently

### **The golden rule for applying operations on entire columns or data frames is to never use loops**

Think about arrays as vectors and the whole data frame as a matrix

If you want to perform any mathematical operation on one or more columns, there is a good chance that the operation is vectorized in Pandas.

For example, the built-in Python operators like +, -, *, /, ** work just like on vectors.

To get a taste of vectorization, let’s perform some operations on a massive dataset. We will choose ~1M row dataset of the old Kaggle TPS September competition:

# Datatable

Datatable is a python library for manipulating tabular data.


It supports out-of-memory datasets, multi-threaded data processing, and flexible API.



The datatable module emphasizes speed and big data support (an area that pandas struggles with); it also has an expressive and concise syntax, which makes datatable also useful for small datasets.

Note: in pandas, there are two fundamental data structures: Series and DataFrame.

In [ ]:
!python3 -m pip install -U pip
!python3 -m pip install -U datatable

In [ ]:
import datatable as dt

tps = dt.fread("/content/train.csv").to_pandas()
tps.shape

The fastest built-in iterator of Pandas is apply.

In [ ]:
def crazy_function(col1, col2, col3):
    return np.sqrt(col1 ** 3 + col2 ** 2 + col3 * 10)

Time the crazy_function on three columns using apply

In [ ]:
%time tps['f1000'] = tps.apply(lambda row: crazy_function(row['f1'], row['f56'], row['f44']), axis=1)

Watch what happens when we pass columns as vectors rather than scalars. No need to modify the function:

In [ ]:
%time tps['f1001'] = crazy_function(tps['f1'], tps['f56'], tps['f44'])

About 600 times faster than the fastest iterator. But we can do even better — vectorization is even faster when used on NumPy arrays:

Add .values to get the underlying NumPy ndarray of Pandas Series.

NumPy arrays are faster because they don't perform additional calls for indexing and data type

In [ ]:
%time tps['f1001'] = crazy_function(tps['f1'].values, tps['f56'].values, tps['f44'].values)

Pandas has a few more tricks up its sleeve.

**Fair warning, though — these won’t benefit you much unless you have upwards of +1M rows.**

In [ ]:
massive_df = pd.concat([tps.drop(["f1000", "f1001"], axis=1)] * 10)
massive_df.shape

In [ ]:
memory_usage = massive_df.memory_usage(deep=True)
memory_usage_in_mbs = np.sum(memory_usage / 1024 ** 2)
memory_usage_in_mbs

Use our crazy_function, start with NumPy vectorization as a baseline


It takes about 0.3 seconds for a 10M row dataset

In [ ]:
%%time

massive_df["f1001"] = crazy_function(
    massive_df["f1"].values, massive_df["f56"].values, massive_df["f44"].values
)

Let’s improve the runtime even more.

The first candidate is Numba.

We install it via pip (pip install numba) and import it. Then, we will decorate our crazy_function with its jit function. JIT stands for just in time, and it translates pure Python and NumPy code to native machine instructions, giving massive speed-ups.

In [ ]:
!pip install numba

In [ ]:
import numba

@numba.jit
def crazy_function(col1, col2, col3):
    return (col1 ** 3 + col2 ** 2 + col3 * 10) ** 0.5

In [ ]:
%%time

massive_df["f1001"] = crazy_function(massive_df["f1"].values, massive_df["f56"].values, massive_df["f44"].values)


We achieved about 1.5 times speed-up.

**Note that Numba works best with functions that involve many native Python loops, a lot of math, and, even better, NumPy functions and arrays.**

# The eval the function of Pandas

There are two versions -
> pd.eval (higher-level)

> df.eval (in the context of DataFrames).

**Like Numba, you should have at least +10,000 samples in the DataFrame to see improvements. But once you do, you will see sizeable benefits in speed.**

Let’s run our crazy_function in the context of df.eval:

In [ ]:
%%time

massive_df.eval("f1001 = (f1 ** 3 + f56 ** 2 + f44 * 10) ** 0.5", inplace=True)


It's not as fast as vectorization or Numba, but it has several benefits. First, you write much less code by avoiding references to the DataFrame name. Next, it significantly speeds up non-math operations on DataFrames like boolean indexing, comparisons, and many more.



### **When you are not doing mathematical manipulation, evaluate your statements in pd.eval.**